# Acquisition Image (raw and cropped)

### Packages

In [ ]:
import numpy as np
import cv2 as cv
from cv2 import VideoCapture as cap
import matplotlib.pyplot as plt
from skimage import io
import os
import math
import time

### Define function

In [ ]:
def write_imgs(from_directory, to_directory, init_number = 1,
              classifier = 'haarcascade_frontalface_default.xml', scale_factor = 1.1, minNeighbors = 5,
              size = (224, 224), color = "RGB"):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    init_number = first number to start naming photos
    classifier = face detector pre-trained (XML file)
    scale_factor = value for classifier (from (1,inf), low values increase false positives)
    minNeighbors = value for classifier (high values decrease false positives)
    size = tuple, format of the saved image
    color = RGB or BW (to save)
    
    '''
    # viene assegnato il classificatore
    face_cascade = cv.CascadeClassifier(classifier)
    
    # variabili per conteggio ed errori
    flag = init_number
    error = 0
    
    for filename in os.listdir(from_directory):
        # check per leggere solamente file jpg o jpeg
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            # lettura dell'immagine            
            img = cv.imread(os.path.join(from_directory, filename))
            # conversione immagine necessaria per il face detector
            gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
            # esecuzione face detector
            faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)
            # check necessario per assegnare solo immagini in cui vengono trovati volti (altrimenti doppioni a catena!)
            if len(faces) == 1:
                for (x, y, w, h) in faces:
                    # ritaglio dell'immagine rispetto al "quadrato" trovato
                    roi_color = img[y:y+h, x:x+w]
                    # resize dell'immagine
                    n_img = cv.resize(roi_color, size)
                    
                    # salvataggio immagine
                    if color == "RGB":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"_rgb.jpg", n_img)
                        flag += 1
                    elif color == "BW":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"_bw.jpg", cv.cvtColor(n_img, cv.COLOR_RGB2GRAY))
                        flag += 1
                        
            # se non trova volti, prosegue qui            
            else:
                error += 1
                try:
                    # delete immagine per non creare doppioni (more safe!)
                    del faces, n_img
                # necessaria se non ha trovato volti (evitare crash code)
                except:
                    pass
                
    # output finale           
    print('Created {}/{} images cropped'.format(flag-1, flag+error-1))
                    
    

In [ ]:
def acquisition_cam(directory, subject, init_number, counter_limit,
                    resolution = (1280, 720), fps = 30, time_sleep = 2, numbered = False):
    '''
    This function allow to access to pc webcam in order to acquire images of the subject.
    
    @params:
    directory = where to save the captured images
    subject = name of the person who is in photo
    init_number = first number to start naming photos
    counter_limit = if desired, limit the number of photos captured
    resolution = (width, height)
    fps = Frame Per Second
    time_sleep = sleep between two photos (second)
    numbered = Bool, necessary in order to apply counter_limit
    '''
    # assegnazione cam (0, prima della lista)
    cam = cv.VideoCapture(0)
    
    # setup configuration cam (resolution, fps)
    cam.set(cv.CAP_PROP_FRAME_WIDTH, resolution[0])
    cam.set(cv.CAP_PROP_FRAME_HEIGHT, resolution[1])
    cam.set(cv.CAP_PROP_FPS, fps)
    # rename windows
    cv.namedWindow("acquisition window")
    
    # variabili per conteggi
    img_counter = init_number
    counter = 1
    
    # ciclo acquisizione frameByframe
    while True:
        # lettura frame e assegnazione alla variabile
        ret, frame = cam.read()
        # riproduco frame per averlo in live (sulla finestra aperta)
        cv.imshow("acquisition window", frame)
        
        # se non va, esco dal ciclo senza rompere code
        if not ret:
            break
        k = cv.waitKey(1)
        
        # utile per impostare un frame rate di acquisizione manuale in secondi (to improve)
        time.sleep(time_sleep)

        # ESC to exit from webcam
        if k%256 == 27:
            print("Escape hit, closing...")
            break

        # fase di salvataggio frame
        else:
            # assegnazione del nome
            img_name = "{}{} ({}).jpg".format(directory, subject, img_counter)
            # salvataggio file
            cv.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            # conteggio per numerazione file
            img_counter += 1
            
            # check per limitare acquisizione
            if numbered:
                if counter == counter_limit:
                    print("Counter limit reached:{}".format(counter_limit))
                    break
                counter+=1

    # chiusura cam e windows cam
    cam.release()
    cv.destroyAllWindows()

### Immagini Ale che sorride :) *example*

In [ ]:
from_directory = "images/raw_images/happy/"
to_directory = "images/happy/"

In [ ]:
# acquisition_cam(directory=from_directory, subject="happy", init_number = 0,
#                 counter_limit = 100, time_sleep = 0.5, numbered = True)

In [ ]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor=1.2, classifier='haarcascade_frontalface_alt2.xml', minNeighbors=8)